# Objetivo

Aplicar o K Nearest Neighbors Na base toda. 

# Método

A partir do conjunto reduzido de features (39) 100k rows, aplicar:
- fillna <b> OK </b>

- LabelEncoder (ou algo do tipo) <b> OK </b>
- Nearest Neighbors OK
    

# Leitura dos arquivos

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Leitura da lista de features a excluir

# Carregamento Clientes

## Redução no número de features



In [2]:
market = pd.read_csv('../../estaticos_market.csv')
port1 = pd.read_csv('clientes2_merge.csv')

In [3]:
exclude_df = pd.read_csv('exclude_variables_generated_from_profiling.csv')
exclude_variables = exclude_df.values.squeeze().tolist()
len(exclude_variables)


143

In [4]:
port1.shape

(566, 182)

In [5]:
market.drop(columns=exclude_variables, inplace= True)
market.drop(columns=['Unnamed: 0'], inplace= True)
market.set_index('id',inplace=True)
market.shape

(462298, 37)

In [6]:
port1.drop(columns=exclude_variables, inplace= True)
port1.drop(columns=['Unnamed: 0'], inplace= True)
port1.set_index('id',inplace=True)
port1.shape

(566, 37)

In [7]:
#p1.to_csv('estaticos_49_features.csv')

# Managing NaNs


Para usar nos dados do portfólio:
- passar as strings para valor numérico
- passar bool para int
- fazer fillna para todos
- - em categoricas -> other
- - em numericas -> 0

Estamos usando o LabelEncoder

In [8]:
fill_dict = market.dtypes.to_dict()
cat2bool = ['fl_optante_simei','fl_optante_simples','fl_passivel_iss']

for feature in fill_dict:
    if(fill_dict[feature] == bool):
        fill_dict[feature] = False
    elif(fill_dict[feature] == object):
        fill_dict[feature] = 'other'
    else:
        fill_dict[feature] = 0
    if feature in cat2bool:
        fill_dict[feature] = False
        
fill_dict

{'fl_matriz': False,
 'de_natureza_juridica': 'other',
 'sg_uf': 'other',
 'natureza_juridica_macro': 'other',
 'de_ramo': 'other',
 'setor': 'other',
 'idade_empresa_anos': 0,
 'idade_emp_cat': 'other',
 'fl_me': False,
 'fl_sa': False,
 'fl_mei': False,
 'fl_ltda': False,
 'dt_situacao': 'other',
 'fl_st_especial': False,
 'fl_email': False,
 'fl_telefone': False,
 'fl_rm': 'other',
 'nm_divisao': 'other',
 'nm_segmento': 'other',
 'fl_optante_simples': False,
 'fl_optante_simei': False,
 'sg_uf_matriz': 'other',
 'de_saude_tributaria': 'other',
 'de_saude_rescencia': 'other',
 'nu_meses_rescencia': 0,
 'de_nivel_atividade': 'other',
 'empsetorcensitariofaixarendapopulacao': 0,
 'nm_meso_regiao': 'other',
 'nm_micro_regiao': 'other',
 'fl_passivel_iss': False,
 'qt_socios': 0,
 'idade_media_socios': 0,
 'qt_socios_feminino': 0,
 'de_faixa_faturamento_estimado': 'other',
 'de_faixa_faturamento_estimado_grupo': 'other',
 'vl_faturamento_estimado_aux': 0,
 'vl_faturamento_estimado_grupo

In [9]:
market.fillna(value=fill_dict, inplace=True)
market[cat2bool] = market[cat2bool].astype(bool)

bool2numeric = market.columns[market.dtypes == bool]
market[bool2numeric] = market[bool2numeric].astype(int)

In [10]:
#market_p1.head(2)

## Encoding

In [11]:
cat = market.columns[market.dtypes == object]
not_cat = market.columns[market.dtypes != object]

In [12]:
# Usando LabelEncoder
#https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/
from sklearn import preprocessing
# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

In [13]:
market_le = pd.DataFrame()
market_le[cat] = market[cat].apply(lambda col: le.fit_transform(col))
market_encoded = pd.concat([market[not_cat],market_le], axis =1)
market_encoded.dtypes

fl_matriz                                  int64
idade_empresa_anos                       float64
fl_me                                      int64
fl_sa                                      int64
fl_mei                                     int64
fl_ltda                                    int64
fl_st_especial                             int64
fl_email                                   int64
fl_telefone                                int64
fl_optante_simples                         int64
fl_optante_simei                           int64
nu_meses_rescencia                       float64
empsetorcensitariofaixarendapopulacao    float64
fl_passivel_iss                            int64
qt_socios                                float64
idade_media_socios                       float64
qt_socios_feminino                       float64
vl_faturamento_estimado_aux              float64
vl_faturamento_estimado_grupo_aux        float64
de_natureza_juridica                       int64
sg_uf               

Aqui fiz fiz um novo dataset para os ids do portifolio1 e o merge com os dados já aplicados o LabelEncoder

In [14]:
port1_le = pd.DataFrame(index=port1.index)
port1_le = market_encoded.merge(port1_le, how='right',left_index=True,right_index=True)


In [15]:
port1_le.head(3)

,fl_matriz,idade_empresa_anos,fl_me,fl_sa,fl_mei,fl_ltda,fl_st_especial,fl_email,fl_telefone,fl_optante_simples,...,nm_divisao,nm_segmento,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,nm_meso_regiao,nm_micro_regiao,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo
id,,,,,,,,,,,,,,,,,,,,,
09e95c1a84049001d086470a8f320a19b076f955a89122ff8360ac2e4d6eca5d,1,11.243836,0,0,0,0,0,1,1,0,...,0,0,0,4,0,0,17,57,6,6
dc9d155f5bcd3172031e2ec9431786e8871ff4b0ff4728bb62194ca0842cce3e,1,43.789041,0,0,0,0,0,0,0,0,...,0,0,19,4,0,0,0,1,4,4
16843c9ffb920170477118933798510d8d5f00f5c56c1221773356355bded321,1,16.389041,0,0,0,0,0,0,1,0,...,36,11,16,4,0,0,9,11,3,3


# Nearest Neighbors em uma amostra do banco

Como não temos o que prever, usamos o treinamento não supervisionado do Nearest Neighbors



Apenas amostra por enquanto

Precisa escolher o numero de componentes a indicar

In [16]:
market_sample = market_encoded.sample(n=50000)

In [17]:
from sklearn.neighbors import NearestNeighbors

nearest = NearestNeighbors(n_neighbors=2, metric='cosine',radius = 5)

nearest.fit(market_sample)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=2, p=2, radius=5)

Dict para a lista de vizinhos mais próximos

In [18]:
neighbors_list = {}

for row in range(port1_le.shape[0]):
    #print(row)
    neighbors_list[row] = nearest.kneighbors(port1_le.iloc[[row]].values)

A resposta é uma lista da distância e uma lista de componentes. A componente é a linha do Dataset, ou seja, clientes.

In [19]:
neighbors_list

{0: (array([[3.52395890e-12, 3.53217455e-12]]), array([[48853, 14301]])),
 1: (array([[2.50133247e-13, 1.48692170e-12]]), array([[23149, 28606]])),
 2: (array([[4.87887508e-12, 1.78626336e-10]]), array([[4894,  690]])),
 3: (array([[4.20885549e-13, 6.11510842e-13]]), array([[23149, 48853]])),
 4: (array([[2.19280150e-12, 1.75311321e-10]]), array([[4894,  690]])),
 5: (array([[8.00403077e-11, 1.08876796e-10]]), array([[25148,  5588]])),
 6: (array([[6.65022482e-11, 1.14072529e-10]]), array([[4216, 5793]])),
 7: (array([[5.89663231e-06, 8.01700902e-06]]), array([[34865, 14277]])),
 8: (array([[7.45498074e-09, 9.39175215e-09]]), array([[ 1919, 24694]])),
 9: (array([[8.38471514e-11, 9.37018241e-11]]), array([[  690, 24191]])),
 10: (array([[6.65179023e-12, 8.52340420e-12]]), array([[19101,  6862]])),
 11: (array([[6.33877983e-07, 7.57361238e-07]]), array([[ 1284, 29525]])),
 12: (array([[3.49561491e-11, 4.39855929e-11]]), array([[37200,  3212]])),
 13: (array([[1.26610833e-10, 1.71590520e

## Avaliação

Agora com o NN funcionando, precisa confrontar as sugestões para saber qual é o mais sugerido.

## Verificando o que foi mais sugerido

Da lista de vizinhos, listar em ordem os mais frequentes.

In [20]:
neighbors_list

{0: (array([[3.52395890e-12, 3.53217455e-12]]), array([[48853, 14301]])),
 1: (array([[2.50133247e-13, 1.48692170e-12]]), array([[23149, 28606]])),
 2: (array([[4.87887508e-12, 1.78626336e-10]]), array([[4894,  690]])),
 3: (array([[4.20885549e-13, 6.11510842e-13]]), array([[23149, 48853]])),
 4: (array([[2.19280150e-12, 1.75311321e-10]]), array([[4894,  690]])),
 5: (array([[8.00403077e-11, 1.08876796e-10]]), array([[25148,  5588]])),
 6: (array([[6.65022482e-11, 1.14072529e-10]]), array([[4216, 5793]])),
 7: (array([[5.89663231e-06, 8.01700902e-06]]), array([[34865, 14277]])),
 8: (array([[7.45498074e-09, 9.39175215e-09]]), array([[ 1919, 24694]])),
 9: (array([[8.38471514e-11, 9.37018241e-11]]), array([[  690, 24191]])),
 10: (array([[6.65179023e-12, 8.52340420e-12]]), array([[19101,  6862]])),
 11: (array([[6.33877983e-07, 7.57361238e-07]]), array([[ 1284, 29525]])),
 12: (array([[3.49561491e-11, 4.39855929e-11]]), array([[37200,  3212]])),
 13: (array([[1.26610833e-10, 1.71590520e

In [21]:
list_size = len(neighbors_list)
num_neighbors = len(neighbors_list[0][1][0])

neighbors_idx_array = neighbors_list[0][1][0]
neighbors_distance_array = neighbors_list[0][0][0]
for line in range(1,list_size):
    neighbors_idx_array = np.concatenate((neighbors_idx_array, neighbors_list[line][1][0]), axis=None) 
    neighbors_distance_array = np.concatenate((neighbors_distance_array, neighbors_list[line][0][0]), axis=None) 

if len(neighbors_idx_array) != list_size*num_neighbors:
    print("ERROR: Check array size.")
# Temos agora um array unidimensional com os índices dos clientes recomendados

In [22]:
neighbors_distance_array[1]

3.5321745528449355e-12

Criar um dicionário com o número da linha, id e distância

In [23]:
dicio = {}
for idx,ind in zip(neighbors_idx_array, range(len(neighbors_idx_array))):
    dicio[ind] = (idx, market_sample.iloc[idx].name, neighbors_distance_array[ind])

In [24]:
dicio

{0: (48853,
  '2d7582cbe03c04e55204ea5523757051c51849b7093664b1d65a003bf37d02f6',
  3.5239589024627094e-12),
 1: (14301,
  'c24a6187673df4395d38fb1f16dce0f396db160d208b1d8cd5caf5c023fd1c69',
  3.5321745528449355e-12),
 2: (23149,
  '70f5bd78df544a353145a1bf8445abe15e9844e4ac0846c8ccc8ed40fd428963',
  2.5013324744804777e-13),
 3: (28606,
  '15f167ec32faf53eaf280016667c946a71ff0ad3820069aa7a50d20e0a6584d5',
  1.4869216968804722e-12),
 4: (4894,
  '4fa4676623d1ee3f10024daece0680ebc41ec1fd131847101b35ce19fdd6037c',
  4.8788750817152504e-12),
 5: (690,
  'dcdabcef5d815e5e6e5bab3a553811d1ad9e628d61e91d3cbbbf0f95afe7ad1b',
  1.7862633594489807e-10),
 6: (23149,
  '70f5bd78df544a353145a1bf8445abe15e9844e4ac0846c8ccc8ed40fd428963',
  4.2088554863539684e-13),
 7: (48853,
  '2d7582cbe03c04e55204ea5523757051c51849b7093664b1d65a003bf37d02f6',
  6.115108419635362e-13),
 8: (4894,
  '4fa4676623d1ee3f10024daece0680ebc41ec1fd131847101b35ce19fdd6037c',
  2.1928014959371467e-12),
 9: (690,
  'dcdabcef5d8

In [25]:
neig_df = pd.DataFrame.from_dict(dicio,orient='index')
neig_df.rename(columns={0:'row', 1:'id',2:'distance'},inplace = True)
neig_df.set_index('id', inplace=True)
#neig_df.row.value_counts()

In [26]:
conflicts = neig_df.merge(port1, how='inner',left_index=True,right_index=True)
suggestion_with_conflicts = neig_df.merge(market, how='left',left_index=True,right_index=True)
#neig_df.shape, port1.shape, conflicts.shape, suggestion_with_conflicts.shape, 
suggestions = suggestion_with_conflicts.drop(conflicts.index)

In [27]:
suggestions

,row,distance,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,...,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,idade_media_socios,qt_socios_feminino,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux
id,,,,,,,,,,,,,,,,,,,,,
00489ba3f0d05b82ff626d66bf32769dff5ba75303bc9b48f2e971e14369de79,7011,8.879453e-12,1,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,SERVICOS DE EDUCACAO,SERVIÇO,17.312329,15 a 20,...,OESTE MARANHENSE,IMPERATRIZ,1,3.0,58.000000,1.0,"DE R$ 30.000.000,01 A R$ 100.000.000,00","DE R$ 30.000.000,01 A R$ 100.000.000,00",58058532.0,58058532.0
00489ba3f0d05b82ff626d66bf32769dff5ba75303bc9b48f2e971e14369de79,7011,2.988609e-12,1,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,SERVICOS DE EDUCACAO,SERVIÇO,17.312329,15 a 20,...,OESTE MARANHENSE,IMPERATRIZ,1,3.0,58.000000,1.0,"DE R$ 30.000.000,01 A R$ 100.000.000,00","DE R$ 30.000.000,01 A R$ 100.000.000,00",58058532.0,58058532.0
00489ba3f0d05b82ff626d66bf32769dff5ba75303bc9b48f2e971e14369de79,7011,5.541567e-12,1,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,SERVICOS DE EDUCACAO,SERVIÇO,17.312329,15 a 20,...,OESTE MARANHENSE,IMPERATRIZ,1,3.0,58.000000,1.0,"DE R$ 30.000.000,01 A R$ 100.000.000,00","DE R$ 30.000.000,01 A R$ 100.000.000,00",58058532.0,58058532.0
00ac2b0f09807d4304312374622e5e6a99337ba14858d30f9f546dc01fc8c5de,12779,2.052691e-11,1,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,PRODUTOS DE AGROPECUARIA,AGROPECUARIA,22.353425,> 20,...,LESTE MARANHENSE,CODO,0,3.0,66.000000,1.0,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",16691184.0,16691184.0
017c5172338c75a2f93d4de00195fbbabf91a54709f9f98674ecb70456f22398,6810,2.790245e-07,0,EMPRESARIO INDIVIDUAL,RN,OUTROS,COMERCIO VAREJISTA,COMERCIO,19.367123,15 a 20,...,CENTRAL POTIGUAR,SERIDO ORIENTAL,0,1.0,69.000000,0.0,"DE R$ 360.000,01 A R$ 1.500.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",1112745.6,2250033.5
022cbff3b74443b07728c3f5ef2d5316f282c9c8444d70cab4fd3ae0619d1ccf,27201,2.503476e-06,0,EMPRESARIO INDIVIDUAL,AM,OUTROS,COMERCIO VAREJISTA,COMERCIO,21.797260,> 20,...,CENTRO AMAZONENSE,MANAUS,0,1.0,0.000000,0.0,"DE R$ 360.000,01 A R$ 1.500.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",989107.2,1545480.0
04632a7f80b5f3b670aedd856420e59483c9135843fd320f23bd0acc144c2964,34865,5.896632e-06,1,SOCIEDADE ANONIMA FECHADA,PI,ENTIDADES EMPRESARIAIS,PRODUTOS DE AGROPECUARIA,AGROPECUARIA,10.594521,10 a 15,...,SUDOESTE PIAUIENSE,FLORIANO,1,3.0,60.500000,0.0,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",11436552.0,14980853.0
04632a7f80b5f3b670aedd856420e59483c9135843fd320f23bd0acc144c2964,34865,1.166450e-06,1,SOCIEDADE ANONIMA FECHADA,PI,ENTIDADES EMPRESARIAIS,PRODUTOS DE AGROPECUARIA,AGROPECUARIA,10.594521,10 a 15,...,SUDOESTE PIAUIENSE,FLORIANO,1,3.0,60.500000,0.0,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",11436552.0,14980853.0
046b94484bbd051fd39a41c4abfff267a1b4f5e07c7ba87ba4bc4998278d8788,8385,8.177103e-08,0,ASSOCIACAO PRIVADA,PI,ENTIDADES SEM FINS LUCRATIVOS,SERVICOS DE SAUDE,SERVIÇO,11.380822,10 a 15,...,CENTRO NORTE PIAUIENSE,TERESINA,1,1.0,36.000000,0.0,"DE R$ 4.800.000,01 A R$ 10.000.000,00","DE R$ 4.800.000,01 A R$ 10.000.000,00",7778916.0,8108618.5


# Visualização dos resultados

## Mapa

### Carrega arquivo com informações geográficas
Gerado por `analysis/Save_IBGE_data_for_Map_Visualization_in_Recommendation_System.ipynb`

In [28]:
coord_micro = pd.read_csv('coord_geograficas_microrregioes_brasil.csv')

In [ ]:
### Importa módulo Folium
import folium
from folium import plugins
import random

### Gera lista com coordenadas das sugestões (suggestions)

In [62]:
map_coord = []
null_count = 0  # para verificar quantas sugestões não tem informação da microrregião
for local in suggestions['nm_micro_regiao']:
    coord = coord_micro[coord_micro['nm_micro'] == local][['lat', 'lng']]
    if not coord.empty:
        map_coord.append([coord['lat'].values[0]-random.uniform(0, 0.25),
                          coord['lng'].values[0]-random.uniform(0, 0.25)])
    else:
        null_count = null_count + 1
        # print('Location null', null_count)   #TODO: mostrar como warning
print('Suggestions without info about nm_micro:', null_count)

Suggestions without info about nm_micro: 88


In [71]:
# Mostra heat map
mapa = folium.Map(location=[-8.788497,-53.879873],tiles='Stamen Toner',zoom_start=5)

mapa.add_child(plugins.HeatMap(map_coord))
mapa

## Lista ordenada por faixa de faturamento